## The Ideal Regenerative Rankine Cycle
Consider a steam power plant operating on the ideal regenerative Rankine cycle with one open feedwater heater. Here are the states:



- (1) after the condenser, entering the pump;
- (2) after the pump, entering the feedwater heater;
- (3) after the feedwater heater, entering the second pump;
- (4) after the second pump, entering the boiler;
- (5) after the boiler, entering the turbine;
- (6) after the turbine, entering the feedwater heater;
- (7) after the turbine, entering the condenser.

Steam enters the high-pressure turbine at $p_5$ and $T_5$.

Steam then enters the feedwater heater at $p_6$.

Steam is then condensed in the condeser at a pressure of $p_7$. 

Determine:
- the fraction of steam extracted from the turbine;
- the thermal efficiency of this cycle.

In [48]:
T5_C = 600
p5_MPa = 15

p6_MPa = 1.2
p7_MPa = 10/1000

# conversion 
T5_K = T5_C + 273.15

### Solution
A steam power plant operates on the ideal regenerative Rankine cycle with one open feedwater heater. The fraction of steam extracted from the turbine and the thermal efficiency are to be determined. 

### Assumptions
- steady operating conditions exist
- kinetic and potential energy changes are negligible 
- as the power plant operates on the ideal Rankine cycle, we assume that the pump and turbine are isentropic; there are no pressure drops in the boiler and condenser; and steam leaves the condenser and feedwater heater and enters the pump as a saturated liquid at the condenser pressure.

In [49]:
x3 = 0
x7 = 0

p1_MPa = p7_MPa

### Analysis
To solve this, we need to know the properties at all of the states. For this, we can use the IAPWS module as our working fluid is water. 

In [50]:
!pip install iapws

IAPWS stands for "The International Association for the Properties of Water and Steam". According to them, they are "...an international non-profit association of national organizations concerned with the properties of water and steam, particularly thermophysical properties, cycle chemistry guidelines, and other aspects of high-temperature steam, water and aqueous mixtures relevant to thermal power cycles and other industrial and scientific applications."

This looks like:

In [51]:
from iapws import IAPWS97

In order to calculate a thermodynamic property, we need to define several properties, such as temperature $T$ and pressure $p$, and for vapor-liquid mixtures, the quality $x$. $T$ should be in the units Kelvin, $p$ should be in MPa, and $x$ should be between 0 and 1.

State 1 is a saturated liquid. Therefore, $x = 0$:

In [52]:
x1 = 0

We then have enough information to calculate all of the thermodynamic properties at this state. The first step in determining properties like $p$, specific entropy $s$, specific enthalpy $h$, etc., is to use the IAPWS97 module. This is written in this case as:

In [53]:
# pressure should be in the units MPa, 
# temperature in the units K, 
# quality a number betwee 0 and 1. 
P1 = p1_MPa
sat_liquid1 = IAPWS97(P=P1, x=x1)

From this, we can now calculate $T_1$ (K), $s_1$ (kJ/(kg K)), $h_1$ (kJ/kg), specific volume $v_1$ (m$^3$/kg).

In [54]:
T1 = sat_liquid1.T # K
s1 = sat_liquid1.s # kJ/kg*K
h1 = sat_liquid1.h # kJ/kg
v1 = sat_liquid1.v # m3/kg

If we want to analyze a steam cycle, we can define three other states. Here we will look at a Rankine cycle, an idealized cycle for a thermoelectric powerplant. So we can say that state 2 is after the compressor, so the pressure is higher ($p_2$), and we will assume that the compressor is isentropic ($s_1 = s_2$):

In [55]:
p2_MPa = p6_MPa
P2 = p2_MPa        # MPa
s2 = s1            # kJ/kg*K

Again, we can calculate the properties at this state as:

In [56]:
sat_liquid = IAPWS97(P=P2, s=s1)
T2 = sat_liquid2.T # K
h2 = sat_liquid2.h # kJ/kg

With the values of $p_1$, $p_2$, and $v_1$ known, we can calculate the specific work, $w_{12}$ (kJ/kg) done by this compressor as:

In [57]:
w12 = v1*(P2 - P1)*1e6/1e3 # kJ/kg; 
# Pressure is in MPa units, 
# and we want this to be in kJ/kg, 
# hence the 1e6/1e3 ratio.

Note: 1e6 = 1000000 and 1e3 = 1000

We can print this value as:

In [58]:
print('The compressor work is: {0:1.4f} kJ/kg'.format(w12))

The compressor work is: 1.2022 kJ/kg


We can now define state 3 of our Rankine cycle, which is after the feedwater heater. 

We know $p_3 = p_2 = p_6$.

In [59]:
p3_MPa = p2_MPa
P3 = p3_MPa # definition of isobaric, MPa

Again, we can calculate the properties at this state as:

In [60]:
liquid = IAPWS97(P=P3, x=x3)
T3 = liquid.T
h3 = liquid.h # kJ/kg
s3 = liquid.s # kJ/kg*K
v3 = liquid.v

Determining state 4. We can assume the pump operates isentropically:

In [61]:
s4 = s3

In [62]:
p4_MPa = p5_MPa
P4 = p4_MPa

liquid2 = IAPWS97(P=P4, s=s4)

In [63]:
w34 = v3*(P4 - P3)*1e6/1e3 # kJ/kg; 
print('The compressor work is: {0:1.4f} kJ/kg'.format(w34))

The compressor work is: 15.7113 kJ/kg


The steam properties can be determined as:

In [64]:
P5 = p5_MPa
T5 = T5_K
steam = IAPWS97(P=P5,T=T5)
h5 = steam.h # kJ/kg
s5 = steam.s #kJ/kg/K

In [65]:
s6 = s5

In [66]:
sat_liquid6 = IAPWS97(P=P6, s=s6)

We can calculate the heat transferred to the water $Q_\text{b}$ (kJ/kg), generating steam by taking the difference in specific enthalpy between states 3 and 2:

Next, the high pressure steam passes through the turbine, which is used to generate electricity in a powerplant. Here we assume that the turbine operates isentropically, $s_4 = s_3$. We also assume that the pressure at the exit of the high pressure turbine is the same as the pressure entering the low pressure turbine, $p_4 = p_5$

With these assumptions, we can calculate the thermodynamic properties of state 4 as:

In [67]:
s4 = s3 # isentropic #kJ/kg*K

In [68]:
P4 = p4_MPa # MPa

In [69]:
steam = IAPWS97(P=P4,s=s4)
T4 = steam.T # K
h4 = steam.h # kJ/kg
x4 = steam.x

We can then calculate the specific work generated by the high pressure turbine, $w_{34}$:

In [70]:
w34 = h4 - h3 # work extracted from the expansion, kJ/kg
print('The turbine work is: {0:1.2f} kJ/kg'.format(w34))

The turbine work is: 15.65 kJ/kg


From the T-s diagram, we see that state 4 is a saturated liquid-vapor region. As an aside, we can calculate its quality:

In [71]:
print('The quality of the vapor at state 4 is: {0:1.2f} (-)'.format(x4))

The quality of the vapor at state 4 is: 0.00 (-)


After the reheating stage, we can calculate the properties of the steam.

In [72]:
P5 = p5_MPa
T5 = T5_K

steamReheat = IAPWS97(P=P5, T=T5)
h5 = steamReheat.h # kJ/kg
s5 = steamReheat.s # kJ/kg*K

We assume that the turbine operates isentropically, therefore $s_6 = s_5$:

In [73]:
s6 = s5

The pressure of the steam entering the condenser is given. So with the entropy and pressure at that state defined, we know the properties:

In [74]:
P6 = p6_MPa
sat_liquid6 = IAPWS97(P=P6, s=s6)
h6 = sat_liquid6.h
x6 = sat_liquid6.x
T6 = sat_liquid6.T

The final state can be determined from:

In [75]:
s7 = s5
P7 = p7_MPa
sat_liquid7 = IAPWS97(P=P7, s=s7)
h7 = sat_liquid7.h
x7 = sat_liquid7.x
T7 = sat_liquid7.T

In [76]:
y = (h3- h2)/(h6 - h2)
print('The value of y is: {0:1.2f} (-)'.format(y))

The value of y is: 0.23 (-)


$y$ is the fraction of steam extracted from the turbine, $y = \frac{\dot{m}_6}{\dot{m}_5}$.

The steam enters the condenser where it is cooled, condensing back into a liquid, where is it is then pumped by the compressor back into the boiler. The amount of heat removed by the condenser can be calculated as ($Q_{\text{c}}$):

In [78]:
Qc = (1-y)*(h7 - h1) # Cooling from T4 to T1, kJ/kg
print('The condenser heat duty is {0:1.2f} kJ/kg'.format(Qc))

The condenser heat duty is 1486.61 kJ/kg


In [79]:
Qb = (h5 - h4) # heat required to make the steam, kJ/kg

print('The boiler heat duty is: {0:1.2f} kJ/kg'.format(Qb))

The boiler heat duty is: 2769.16 kJ/kg


The efficiency of the Rankine cycle can be calculated then as:

In [80]:
eta = 1 - Qc/Qb
print('The overall efficiency is {0:1.2%}.'.format(eta))

The overall efficiency is 46.32%.


The negative sign we have in front of the $w$ terms is due to the convention we used here. In our convention, the work we calculated was the energy consumed by the device. Normally, the convention in thermodynamics is that the work done by a system is positive, and the heat transferred into a system is positive. The opposites of this are then negative. For whatever convention you decide to use, the main point is to be consistent, although we recommend following the norm. 

It is instructive to compare the efficiency we calculated here with the Carnot efficiency. Recall that:

$\eta_\text{Carnot} = 1 - \frac{T_\text{C}}{T_\text{H}}$.

In [81]:
TC_K = T1
TH_K = T5

In [82]:
eta_carnot = 1 - TC_K / TH_K

print('The Carnot efficiency of this power plant is {0:1.2%}.'.format(eta_carnot))

The Carnot efficiency of this power plant is 63.47%.
